This notebook is a fork of https://www.kaggle.com/ahmedelhaddad/fastai-submission-custom-ds 

Many thanks to Dr. Ahmed Sami for sharing his model and training flow! The only thing I changed was the output
to a tensor of 5 items, to do BCEWithLogitsLoss https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html 
as well as some of the hyperparameters, ALSO of course I added an trained Encoder layer to the model! 

This helped improve the model in all scenarios, and I will explain my reasoning for this and trials in the discussion forum soon!

I am going to fork into many different models I've tried(here are the links and scores)
https://www.kaggle.com/pyoungkangkim/mlp-batchnorm-dropout-linear-pytorch-no-pre-bnorm
https://www.kaggle.com/pyoungkangkim/resnet-batchnorm-dropout-linear-pytorch-jstreet
https://www.kaggle.com/pyoungkangkim/densenet-batchnorm-dropout-linear-pytorch-jstreet

More to come below, I'll eventually release them(soon) once I submit them and get the scores.
https://www.kaggle.com/pyoungkangkim/1dcnn-pytorch-jstreet
https://www.kaggle.com/pyoungkangkim/resnet-deep-bnorm-dropout-linear-pytorch-jstreet

The particulary interesting ones to come(I speculate sometime mid Feburary) will be ensemble of 
these models and xgboost, check later on my profile if you're interested for this and trials/findings! 


In [ ]:
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import datatable as dtable
import fastai

from fastai.tabular.all import *
from fastai.data.core import DataLoader
from fastai.data.core import DataLoaders
from numba import njit
from tqdm import tqdm
from sklearn.model_selection import cross_val_score
from time import time

df = dtable.fread('/kaggle/input/jane-street-market-prediction/train.csv').to_pandas()
df = df.query('date > 85').reset_index(drop = True) 
df = df[df.weight > 0]
df.reset_index(inplace=True, drop=True)
df = df.astype({c: np.float32 for c in df.select_dtypes(include='float64').columns})
df_labels = df[['date', 'weight', 'resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp']]
features = [c for c in df.columns if 'feature' in c]
df = df.drop(df_labels.columns, axis=1)
df.fillna(df.mean(), inplace=True)
resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp']
X = df[features]
y = np.stack([(df_labels[c] > 0).astype('int') for c in resp_cols]).T
f_mean = np.mean(df[features[1:]].values, axis=0)
device = 'cuda' if t.cuda.is_available() else 'cpu'

In [ ]:
encoder_out_dim=64

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(nn.BatchNorm1d(130),
                                    LinBnDrop(130, 256, bn=True, p=0, act=Mish(), lin_first=False),
                                    LinBnDrop(256, 512, bn=True, p=0.12, act=Mish(), lin_first=False),   
                                    LinBnDrop(512, 256, bn=True, p=0.12, act=Mish(), lin_first=False),
                                    LinBnDrop(256, encoder_out_dim, bn=False, act=None, lin_first=False))
        
        self.decoder = nn.Sequential(nn.BatchNorm1d(64),
                                    LinBnDrop(64, 400, bn=True, p=0, act=Mish(), lin_first=False),
                                    LinBnDrop(400, 800, bn=True, p=0.12, act=Mish(), lin_first=False),   
                                    LinBnDrop(800, 400, bn=True, p=0.12, act=Mish(), lin_first=False),
                                    LinBnDrop(400, 130, bn=False, act=None, lin_first=False))
        
    def encode(self, x):
        return self.encoder(x)
    def decode(self, x):
        return self.decoder(x)
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
ae = Learner(None, Autoencoder(), loss_func=1)
ae.load('/kaggle/input/jstreetae1/autoencoder_rmse')

In [ ]:
class Model(nn.Module):

    def __init__(self, input_dim):
        super(Model, self).__init__() 
        self.encoder = ae.model.encoder
        self.encoder.eval()
        self.layers = nn.Sequential(nn.BatchNorm1d(input_dim),
                                    LinBnDrop(input_dim, 400, bn=True, p=0, act=Mish(), lin_first=False),
                                    LinBnDrop(400, 800, bn=True, p=0.2289, act=Mish(), lin_first=False),   
                                    LinBnDrop(800, 400, bn=True, p=0.2289, act=Mish(), lin_first=False),
                                    LinBnDrop(400, 5, bn=False, act=None, lin_first=False)) 

    def forward(self, x):
        encoded_x = self.encoder(x).detach()
        x = t.cat((encoded_x, x), dim=1)
        x = self.layers(x)
        return x

    


In [ ]:
model_nn = Model(len(features) + encoder_out_dim)
model_nn = model_nn.to(device)
learn = Learner(None, model_nn, loss_func=1)
learn.load('/kaggle/input/jstreetmlpae1/ae_roc')

In [ ]:
@njit
def fillna_npwhere_njit(array, values):
    if np.isnan(array.sum()):
        array = np.where(np.isnan(array), values, array)
    return array

def for_loop(method, matrix, values):
    for i in range(matrix.shape[0]):
        matrix[i] = method(matrix[i], values)
    return matrix

In [ ]:
%%time
%%capture
import janestreet
env = janestreet.make_env()

learn.model.eval()
preds = []
for (test_df, pred_df) in tqdm(env.iter_test()):
    if test_df['weight'].item() > 0:   
        x_tt = test_df.loc[:, features].values
        x_tt[:, 1:] = for_loop(fillna_npwhere_njit, x_tt[:, 1:], f_mean)
        pred = 0.
        pred = learn.model(t.from_numpy(x_tt).to(device, t.float)).sigmoid()[0][-1].item()
        preds.append(pred)
        action = 1 if pred >= .5 else 0
        pred_df.action = action
    else:
        pred_df.action = 0
    env.predict(pred_df)

In [ ]:
preds = np.array(preds)
preds.mean(), preds.std(), sum(preds >= .5), sum(preds < 5)